In [ ]:
%env DATA_BUCKET=tbd-2023z-9910-data
%env GEN_OUTPUT_DIR=/tmp/tpc-di
%env REPO_ROOT=/home/jupyter/git/tbd-tpc-di/

In [13]:
!pip install typer[All]==0.9.0 google-cloud-storage==2.13.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 42.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboardx 2.6 requires protobuf<4,>=3.8.0, but you have protobuf 4.25.1 which is incompatible.


## Install SDKMAN for setting up JVM 8 enviroment

In [ ]:
%%bash
curl -s https://get.sdkman.io | bash

## Install and set as default JVM 8

In [ ]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
sdk install java 8.0.392-amzn
sdk use java 8.0.392-amzn

## Check if JVM 8 is used

In [ ]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
java -version

## Clone tbd-tpc-di repo

In [ ]:
%%bash
mkdir -p git && cd git
git clone https://github.com/mwiewior/tbd-tpc-di.git
cd tbd-tpc-di
git checkout feat/labs
git pull

## Generate input dataset (run this cell below from the terminal!!!)
It should take approx. 15min with scale factor set to 100 and generate approx. 10GiB

In [ ]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
cd $REPO_ROOT/tools/
java -jar DIGen.jar -sf 100 -o /tmp/tpc-di

In [ ]:
# Install and setup JVM 11

In [ ]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
sdk install java 11.0.21-amzn
sdk use java 11.0.21-amzn

## Load staging

In [ ]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
cd $REPO_ROOT
python tpcdi.py --output-directory $GEN_OUTPUT_DIR --stage $DATA_BUCKET

## Run dbt ELT

In [5]:
%%bash
cd $REPO_ROOT/tools/
dbt run

00:16:36  Running with dbt=1.7.3
00:16:37  Registered adapter: spark=1.7.1
00:16:37  [WARNING]: Configuration paths exist in your dbt_project.yml file which do not apply to any resources.
There are 4 unused configuration paths:
- models.dbt_tpcdi.bronze
- models.dbt_tpcdi.silver
- models.dbt_tpcdi.gold
- models.dbt_tpcdi.work
00:16:37  Found 45 models, 1 test, 17 sources, 0 exposures, 0 metrics, 553 macros, 0 groups, 0 semantic models
00:16:37  


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/12/04 00:16:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/04 00:16:52 WARN HiveClientImpl: Detected HiveConf hive.execution.engine is 'tez' and will be reset to 'mr' to disable useless hive logic
00:16:55  Concurrency: 1 threads (target='dev')
00:16:55  
00:16:55  1 of 45 START sql view model bronze.brokerage_cash_transaction ................. [RUN]
23/12/04 00:16:56 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
23/12/04 00:16:57 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
00:16:57  1 of 45 OK created sql view model bronze.brokerage_cash_transaction ............ [OK in 1.99s]
00:16:57  2 of 45 START sql view model bronze.brokerage_daily_market ..................... [RUN]
00:16:58  2 of 45 OK created sql view model bronze.brokerag

## Run dbt tests

In [7]:
%%bash
cd $REPO_ROOT/tools/
dbt test

00:19:46  Running with dbt=1.7.3
00:19:47  Registered adapter: spark=1.7.1
00:19:48  [WARNING]: Configuration paths exist in your dbt_project.yml file which do not apply to any resources.
There are 4 unused configuration paths:
- models.dbt_tpcdi.gold
- models.dbt_tpcdi.bronze
- models.dbt_tpcdi.work
- models.dbt_tpcdi.silver
00:19:48  Found 45 models, 1 test, 17 sources, 0 exposures, 0 metrics, 553 macros, 0 groups, 0 semantic models
00:19:48  


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/12/04 00:19:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/04 00:20:01 WARN HiveClientImpl: Detected HiveConf hive.execution.engine is 'tez' and will be reset to 'mr' to disable useless hive logic
00:20:06  Concurrency: 1 threads (target='dev')
00:20:06  
00:20:06  1 of 1 START test fact_trade__unique_trade ..................................... [RUN]
23/12/04 00:20:08 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
00:21:45  1 of 1 PASS fact_trade__unique_trade ........................................... [PASS in 98.69s]
00:21:45  
00:21:45  Finished running 1 test in 0 hours 1 minutes and 57.07 seconds (117.07s).
00:21:45  
00:21:45  Completed successfully
00:21:45  
00:21:45  Done. PASS=1 WARN=0 ERROR=0 SKIP=0 TOTAL=1


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TBD-TPC-DI-setup") \
    .master("yarn") \
    .enableHiveSupport() \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/12/04 00:26:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/04 00:26:38 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
23/12/04 00:26:38 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
spark.sql("show databases").show()

23/12/04 00:27:49 WARN HiveClientImpl: Detected HiveConf hive.execution.engine is 'tez' and will be reset to 'mr' to disable useless hive logic


,namespace
0,bronze
1,default
2,digen
3,gold
4,silver


In [10]:
spark.sql("use demp_gold")

DataFrame[]

In [11]:
spark.sql("show tables").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|   bronze|            accounts|      false|
|   bronze|brokerage_cash_tr...|      false|
|   bronze|brokerage_daily_m...|      false|
|   bronze|brokerage_holding...|      false|
|   bronze|     brokerage_trade|      false|
|   bronze|brokerage_trade_h...|      false|
|   bronze|brokerage_watch_h...|      false|
|   bronze|   cash_transactions|      false|
|   bronze|           companies|      false|
|   bronze|   crm_customer_mgmt|      false|
|   bronze|           customers|      false|
|   bronze|        daily_market|      false|
|   bronze|                date|      false|
|   bronze|         dim_account|      false|
|   bronze|          dim_broker|      false|
|   bronze|         dim_company|      false|
|   bronze|        dim_customer|      false|
|   bronze|            dim_date|      false|
|   bronze|        dim_security|      false|
|   bronze

In [14]:
spark.stop()